In [17]:
!pip install -q --upgrade huggingface_hub

In [1]:
# accelerate==0.29.3
# datasets==2.19.0
# diffusers==0.28.0
# einops==0.8.0
# Pillow==10.3.0
# torch==2.2.0
# torchvision==0.17.0
# tqdm==4.66.2
# transformers==4.40.1

!pip install pillow torchmetrics torchmetrics[image] torch-fidelity
!pip install -q --upgrade datasets fsspec
#  or, if you want to pin exactly the fixed version:
# !pip install -q datasets==2.14.7 fsspec==2023.9.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 961.5/961.5 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 127.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 98.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 63.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 43.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 103.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Unins

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [65]:
rm -rf data

Here iss the formatted Markdown table:

Script	Unicode	Character	Caption
Latin	U+0041	A	“A”
	U+0042	B	“B”
	U+0044	D	“D”
	U+0047	G	“G”
	U+0057	W	“W”
Arabic	U+0641	ف	“faa”
	U+0642	ق	“qaaf”
	U+063A	غ	“ghayn”
	U+0644	ل	“laam”
	U+0645	م	“meem”
Devanagari	U+0915	क	“ka”
	U+0932	ल	“la”
	U+0950	ॐ	“om”
	U+0917	ग	“ga”
	U+092E	म	“ma”

# Get Arabic Characters

In [66]:
import os
from kaggle.api.kaggle_api_extended import KaggleApi # run this cell twice if the import fails
import zipfile

# ─── 1) Configure your Kaggle credentials in-process ──────────────────────────────
os.environ['KAGGLE_USERNAME'] = 'adityaks99'       # replace with your username
os.environ['KAGGLE_KEY']      = 'c5cb1c09b368431d2e753f573b91cc79'  # your API key

# ─── 2) Authenticate to Kaggle ───────────────────────────────────────────────────
api = KaggleApi()
api.authenticate()  # raises if creds are invalid :contentReference[oaicite:0]{index=0}

# ─── 3) Download the dataset ZIP and auto-unzip ──────────────────────────────────
dataset_slug = 'hossammbalaha/hmbd-v1-arabic-handwritten-characters-dataset'
target_dir   = 'HMBD_v1'  # local directory to hold PNGs

# This fetches and extracts all files into `target_dir/` :contentReference[oaicite:1]{index=1}
api.dataset_download_files(
    dataset_slug,
    path=target_dir,
    unzip=True,
    force=True,   # overwrite existing files if re-running
    quiet=False
)

# ─── 4) (Optional) Verify contents ───────────────────────────────────────────────
# After the above, you should see subfolders named by class, each containing 300×300 PNGs.
for root, dirs, files in os.walk(target_dir):
    if files:
        print(f'Found {len(files)} files in {root}')
        break


Dataset URL: https://www.kaggle.com/datasets/hossammbalaha/hmbd-v1-arabic-handwritten-characters-dataset


100%|██████████| 169M/169M [00:00<00:00, 1.32GB/s]

Found 4 files in HMBD_v1


In [67]:
#!/usr/bin/env python3
"""
Automated pipeline to:
1. Enumerate all Arabic‐script Unicode code points
2. Generate a CSV with Unicode, character, and name
3. Download a set of up-to-date, canonical Arabic fonts
4. Incorporate a locally-downloaded Scheherazade ZIP
5. Extract all TTFs
6. Render each character in each font to 128×128 PNGs
"""

import unicodedata
import csv
import logging
from pathlib import Path
import requests
import zipfile
from PIL import Image, ImageDraw, ImageFont

# -----------------------------------------------------------------------------
# 1) Configuration
# -----------------------------------------------------------------------------

# Unicode blocks covering Arabic script, supplements, and presentation forms
UNICODE_BLOCKS = [
    (0x0600, 0x06FF),   # Arabic
    (0x0750, 0x077F),   # Arabic Supplement
    (0x08A0, 0x08FF),   # Arabic Extended-A
    (0xFB50, 0xFDFF),   # Arabic Presentation Forms-A
    (0xFE70, 0xFEFF),   # Arabic Presentation Forms-B
]

# Fonts to download (static TTFs)
FONTS = {
    "NotoNaskhArabic[wght].ttf":
        "https://github.com/google/fonts/raw/main/ofl/notonaskharabic/NotoNaskhArabic%5Bwght%5D.ttf",
    "Amiri-Regular.ttf":
        "https://github.com/google/fonts/raw/main/ofl/amiri/Amiri-Regular.ttf",
    "Lateef-Regular.ttf":
        "https://github.com/google/fonts/raw/main/ofl/lateef/Lateef-Regular.ttf",
    "Harmattan-Regular.ttf":
        "https://github.com/google/fonts/raw/main/ofl/harmattan/Harmattan-Regular.ttf",
}


# Image settings
IMG_SIZE  = (128, 128)
FONT_SIZE = 100

# -----------------------------------------------------------------------------
# 2) Paths & Logging
# -----------------------------------------------------------------------------

BASE_DIR   = Path.cwd()
DATA_DIR   = BASE_DIR / "data_generation"
FONTS_DIR  = DATA_DIR / "fonts"
CSV_PATH   = DATA_DIR / "arabic_chars.csv"
IMAGES_DIR = DATA_DIR / "images"


# Path to your manually-downloaded Scheherazade ZIP
LOCAL_SCHEHERAZADE_ZIP = Path(
    f"{DATA_DIR}/manually_downloaded_fonts/Scheherazade_New.zip"
)

for d in (DATA_DIR, FONTS_DIR, IMAGES_DIR):
    d.mkdir(parents=True, exist_ok=True)

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
)

# -----------------------------------------------------------------------------
# 3) Step 1: Generate CSV of Arabic characters
# -----------------------------------------------------------------------------

logging.info("Step 1: Generating CSV of Arabic‐script characters...")

with CSV_PATH.open("w", newline="", encoding="utf-8") as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["Unicode", "Character", "Name"])
    for start, end in UNICODE_BLOCKS:
        for cp in range(start, end + 1):
            char = chr(cp)
            name = unicodedata.name(char, "")
            if name.startswith("ARABIC"):
                writer.writerow([f"U+{cp:04X}", char, name])

logging.info(f"  • CSV saved to {CSV_PATH}")

# -----------------------------------------------------------------------------
# 4) Step 2: Download fonts
# -----------------------------------------------------------------------------

logging.info("Step 2: Downloading fonts from the web...")

for fname, url in FONTS.items():
    dest = FONTS_DIR / fname
    if dest.exists():
        logging.info(f"  • {fname} already exists, skipping")
        continue
    logging.info(f"  • Downloading {fname}")
    resp = requests.get(url)
    resp.raise_for_status()
    dest.write_bytes(resp.content)
    logging.info(f"    → Saved to {dest}")

# -----------------------------------------------------------------------------
# 5) Step 3: Incorporate local Scheherazade ZIP
# -----------------------------------------------------------------------------

if LOCAL_SCHEHERAZADE_ZIP.exists():
    logging.info("Step 3: Extracting local Scheherazade ZIP...")
    with zipfile.ZipFile(LOCAL_SCHEHERAZADE_ZIP, 'r') as zf:
        for member in zf.namelist():
            if member.lower().endswith(".ttf"):
                # extract into fonts directory
                target = FONTS_DIR / Path(member).name
                with zf.open(member) as src, target.open('wb') as dst:
                    dst.write(src.read())
                logging.info(f"    → Extracted {target.name}")
else:
    logging.warning(f"Local Scheherazade ZIP not found at {LOCAL_SCHEHERAZADE_ZIP}")

# -----------------------------------------------------------------------------
# 6) Step 4: Render images for each font  – with “glyph-supported” check
# -----------------------------------------------------------------------------
logging.info("Step 4: Rendering images for each font…")

SCRATCH = 256          # temporary big canvas
TARGET  = 128          # final PNG size
MISSING = {}           # font_name → count of unsupported glyphs

def glyph_supported(ch: str, ft: ImageFont.FreeTypeFont) -> bool:
    """Return True if this font has a real outline for *ch* (not fallback □)."""
    tmp = Image.new("L", (64, 64), 255)
    dr  = ImageDraw.Draw(tmp)
    dr.text((0,0), ch, font=ft, fill=0)
    img_char = tmp.tobytes()

    tmp2 = Image.new("L", (64, 64), 255)
    dr2  = ImageDraw.Draw(tmp2)
    dr2.text((0,0), "�", font=ft, fill=0)
    img_repl = tmp2.tobytes()
    return img_char != img_repl

# Load CSV once
with CSV_PATH.open("r", encoding="utf-8") as csvfile:
    records = list(csv.DictReader(csvfile))

# Collect all TTF / OTFs
font_files = sorted(f for f in FONTS_DIR.iterdir()
                    if f.suffix.lower() in {".ttf", ".otf"})

for font_file in font_files:
    font_name = font_file.stem
    out_dir   = IMAGES_DIR / font_name
    out_dir.mkdir(exist_ok=True)

    logging.info(f"  • Rendering with font: {font_name}")
    font = ImageFont.truetype(str(font_file), FONT_SIZE)
    missing = 0

    for row in records:
        code = row["Unicode"]      # e.g. "U+0621"
        char = row["Character"]
        out  = out_dir / f"{font_name}_{code}.png"
        if out.exists():
            continue

        if not glyph_supported(char, font):
            missing += 1
            continue

        # 1. draw glyph centred on big scratch canvas
        scratch = Image.new("L", (SCRATCH, SCRATCH), 255)
        dr      = ImageDraw.Draw(scratch)
        dr.text((SCRATCH/2, SCRATCH/2), char, font=font, fill=0, anchor="mm")

        # 2. tight-crop & 3. scale to 128 keeping aspect
        bbox = scratch.getbbox()
        if not bbox:
            missing += 1
            continue
        glyph = scratch.crop(bbox)
        g_w, g_h = glyph.size
        scale = min(TARGET / g_w, TARGET / g_h)
        glyph = glyph.resize((int(g_w*scale), int(g_h*scale)), Image.LANCZOS)

        # 4. paste on final 128×128 white canvas
        final = Image.new("RGB", (TARGET, TARGET), "white")
        x = (TARGET - glyph.width) // 2
        y = (TARGET - glyph.height) // 2
        final.paste(Image.merge("RGB", (glyph,)*3), (x, y))
        final.save(out)

    total = len(records)
    drawn = total - missing
    MISSING[font_name] = missing
    logging.info(f"    → {drawn}/{total} glyphs rendered "
                 f"({missing} unsupported)")

logging.info("✅ Pipeline complete!")
for fname, miss in MISSING.items():
    logging.info(f"   • {fname}: {miss} unsupported glyphs")
logging.info(f" • CSV:    {CSV_PATH}")
logging.info(f" • Fonts:  {FONTS_DIR}")
logging.info(f" • Images: {IMAGES_DIR}")

In [68]:
from PIL import Image, UnidentifiedImageError
import os

data_root = "/content/HMBD_v1/Dataset"
bad_files = []
removed_dirs = []

# 1) Identify and delete unreadable images
for root, _, files in os.walk(data_root):
    for fname in files:
        path = os.path.join(root, fname)
        try:
            # verify() reads header only; raises if format unrecognized :contentReference[oaicite:0]{index=0}
            with Image.open(path) as img:
                img.verify()
        except (UnidentifiedImageError, IOError):
            bad_files.append(path)
            os.remove(path)  # delete the corrupted/mis-formatted file :contentReference[oaicite:1]{index=1}
            print(f"Removed unreadable image: {path}")

# 2) Remove any now-empty directories (walk bottom-up) :contentReference[oaicite:2]{index=2}
for root, dirs, files in os.walk(data_root, topdown=False):
    # if no files and no subdirs remain, delete this dir
    if not dirs and not files:
        os.rmdir(root)
        removed_dirs.append(root)
        print(f"Removed empty directory: {root}")

# 3) Print summary
print("\n===== Cleanup Summary =====")
print(f"Total unreadable images removed: {len(bad_files)}")
print(f"Total empty directories removed: {len(removed_dirs)}")


Streaming output truncated to the last 5000 lines.
Removed unreadable image: /content/HMBD_v1/Dataset/Khaa_Isolated/AHCR_00064_Khaa_Isolated_22.jpg
Removed unreadable image: /content/HMBD_v1/Dataset/Khaa_Isolated/AHCR_00075_Khaa_Isolated_31.jpg
Removed unreadable image: /content/HMBD_v1/Dataset/Khaa_Isolated/AHCR_00110_Khaa_Isolated_31.jpg
Removed unreadable image: /content/HMBD_v1/Dataset/Khaa_Isolated/AHCR_00014_Khaa_Isolated_31.jpg
Removed unreadable image: /content/HMBD_v1/Dataset/Khaa_Isolated/AHCR_00012_Khaa_Isolated_30.jpg
Removed unreadable image: /content/HMBD_v1/Dataset/Khaa_Isolated/AHCR_00105_Khaa_Isolated_23.jpg
Removed unreadable image: /content/HMBD_v1/Dataset/Khaa_Isolated/AHCR_00086_Khaa_Isolated_30.jpg
Removed unreadable image: /content/HMBD_v1/Dataset/Khaa_Isolated/AHCR_00052_Khaa_Isolated_23.jpg
Removed unreadable image: /content/HMBD_v1/Dataset/Khaa_Isolated/AHCR_00115_Khaa_Isolated_23.jpg
Removed unreadable image: /content/HMBD_v1/Dataset/Khaa_Isolated/AHCR_00120_

In [69]:
# prompt: write out meta

logging.info("✅ Pipeline complete!")
for fname, miss in MISSING.items():
    logging.info(f"   • {fname}: {miss} unsupported glyphs")
logging.info(f" • CSV:    {CSV_PATH}")
logging.info(f" • Fonts:  {FONTS_DIR}")
logging.info(f" • Images: {IMAGES_DIR}")


In [70]:
# %% ──────────────────────────────────────────────────────────────
!pip install -q --upgrade huggingface_hub pillow tqdm

import os, json, zipfile, random, shutil, logging
from pathlib import Path
from PIL import Image, ImageDraw, ImageFont, UnidentifiedImageError
from huggingface_hub import hf_hub_download, list_repo_files

# Data Generation

In [71]:
rm -rf data

In [72]:
# %% ──────────────────────────────────────────────────────────────
# ── dataset layout ──────────────────
OUT_ROOT   = Path("data")          # everything lands here
OUT_ROOT.mkdir(parents=True, exist_ok=True)
IMG_SZ     = 128
KEEP_N     = 100                                # images per glyph
RNG        = random.Random(2024)

# ── character selections ─────────────
ENGLISH  = [("0041","A","A"),
            ("0042","B","B"),
            ("0047","G","G")]

HINDI    = [("0915","ka","ka"),       # क
            ("0932","la","la"),       # ल
            ("0950","ga","om")]       # ॐ

ARABIC   = [("0641","ف","faa","Faa_Isolated"),
            ("0642","ق","qaaf","Qaf_Isolated"),
            ("0645","م","meem","Mem_Isolated")]

# ── remote repos ─────────────────────
HF_EN = "Mayank022/English_Characters_Images"
HF_HI = "Mayank022/Devanagari-Characters-Image"

# ── HMBD + font info for Arabic ──────
HMBD_ROOT = Path("/content/HMBD_v1/Dataset")    # adjust if different
FONTS_DIR = Path("data_generation/fonts")       # four TTFs already downloaded
FONT_TTFS = ["NotoNaskhArabic[wght].ttf",
             "Amiri-Regular.ttf",
             "Lateef-Regular.ttf",
             "Harmattan-Regular.ttf"]

logging.basicConfig(level=logging.INFO,
                    format="%(levelname)s: %(message)s")

In [73]:
# %% ──────────────────────────────────────────────────────────────
def grab_character_images(repo_id: str,
                           char_tag: str,      # "A", "ka", …
                           unicode_hex: str,   # "0041", …
                           out_dir: Path,
                           keep: int = 100,
                           rng: random.Random | None = None,
                           img_sz: int = 128):
    """
    Copy ≤ *keep* 128×128 PNG/JPG images for one glyph from Mayank’s dataset.
    Handles both <char>.zip and extracted <char>/ folder layouts.
    """
    rng = rng or random
    out_dir.mkdir(parents=True, exist_ok=True)
    log = logging.getLogger(f"grab:{char_tag}")

    all_files = list_repo_files(repo_id, repo_type="dataset")
    log.info(f"[{char_tag}] {len(all_files)} entries in {repo_id}")

    # ── look for ZIP first ──
    zips = [f"{char_tag}.zip", f"{char_tag.upper()}.zip", f"{unicode_hex}.zip"]
    hit  = next((z for z in zips if z in all_files), None)

    if hit:                                            # ZIP branch
        zpath = hf_hub_download(repo_id, hit, repo_type="dataset")
        with zipfile.ZipFile(zpath) as zf:
            imgs = [m for m in zf.namelist()
                    if m.lower().endswith((".png", ".jpg", ".jpeg"))]
            rng.shuffle(imgs); imgs = imgs[:keep]
            log.info(f"  • {hit} → {len(imgs)} kept")
            for i, member in enumerate(imgs):
                with zf.open(member) as fh:
                    Image.open(fh).convert("L")\
                         .resize((img_sz, img_sz), Image.LANCZOS)\
                         .save(out_dir / f"{i:03d}.png")
        return

    # ── else: look for extracted folder ──
    prefixes = [f"{char_tag}/", f"{char_tag.upper()}/", f"{unicode_hex}/"]
    prefix   = next((p for p in prefixes if any(f.startswith(p) for f in all_files)), None)
    if not prefix:
        raise FileNotFoundError(f"No ZIP or folder for “{char_tag}” in {repo_id}")

    imgs = [f for f in all_files
            if f.startswith(prefix) and f.lower().endswith((".png",".jpg",".jpeg"))]
    rng.shuffle(imgs); imgs = imgs[:keep]
    log.info(f"  • folder {prefix} → {len(imgs)} kept")
    for i, remote in enumerate(imgs):
        local = hf_hub_download(repo_id, remote, repo_type="dataset")
        Image.open(local).convert("L")\
             .resize((img_sz, img_sz), Image.LANCZOS)\
             .save(out_dir / f"{i:03d}.png")

In [74]:
# %% ──────────────────────────────────────────────────────────────
SCRATCH = 256
def crop_hmbd(img: Image.Image) -> Image.Image:
    w,h = img.size
    s   = min(w,h)
    img = img.crop(((w-s)//2, (h-s)//2, (w+s)//2, (h+s)//2))
    return img.resize((IMG_SZ, IMG_SZ), Image.LANCZOS).convert("L")

def random_affine(im: Image.Image, rng=RNG) -> Image.Image:
    ang = rng.uniform(-7,7); dx,dy = rng.randint(-4,4), rng.randint(-4,4)
    return im.rotate(ang, expand=True, fillcolor=255)\
             .transform(im.size, Image.AFFINE, (1,0,dx, 0,1,dy), fillcolor=255)

def render_font(glyph, ft_path, rng=RNG):
    ft = ImageFont.truetype(str(ft_path), rng.randint(96,118))
    canvas = Image.new("L", (SCRATCH,SCRATCH), 255)
    ImageDraw.Draw(canvas).text((SCRATCH//2,SCRATCH//2), glyph,
                                font=ft, fill=0, anchor="mm")
    canvas = random_affine(canvas, rng)
    if not canvas.getbbox():                     # glyph missing in font
        return None
    g = canvas.crop(canvas.getbbox())
    sc = min(IMG_SZ/g.width, IMG_SZ/g.height)
    g = g.resize((int(g.width*sc), int(g.height*sc)), Image.LANCZOS)
    final = Image.new("L", (IMG_SZ,IMG_SZ), 255)
    final.paste(g, ((IMG_SZ-g.width)//2, (IMG_SZ-g.height)//2))
    return final

In [87]:
meta = []
english_codes = {t[0] for t in ENGLISH}      # {"0041","0042",…}

for uni, char_tag, cap in ENGLISH + HINDI:
    repo = HF_EN if uni in english_codes else HF_HI
    out  = OUT_ROOT / uni
    logging.info(f"{cap}: fetching 100 samples from {repo}")

    grab_character_images(repo, char_tag, uni, out, keep=KEEP_N)

    IMG_EXTS = {".png", ".jpg", ".jpeg"}

    for p in out.iterdir():
        if p.suffix.lower() not in IMG_EXTS or not p.is_file():
            continue                    # ← skip ZIPs, json, cache dirs, …
        meta.append({
            "Unicode":   uni,
            "Character": char_tag,
            "caption":   cap,
            "file_name": f"{uni}/{p.name}"
        })

# Arabic


In [88]:
# %% ──────────────────────────────────────────────────────────────
font_paths = [FONTS_DIR/f for f in FONT_TTFS]
for p in font_paths:
    if not p.exists():
        raise FileNotFoundError(f"Font missing → {p}")

for uni, glyph, cap, sub in ARABIC:
    out_dir = OUT_ROOT/uni
    out_dir.mkdir(exist_ok=True)
    logging.info(f"{cap}: processing HMBD + fonts")

    # -- HMBD real samples --
    src = HMBD_ROOT/sub
    files = [p for p in src.iterdir() if p.suffix.lower() in {".png",".jpg",".jpeg"}]
    RNG.shuffle(files)
    for i, f in enumerate(files[:KEEP_N - 10]):
        try:
            img = crop_hmbd(Image.open(f))
            img.save(out_dir/f"{i:03d}.png")
            meta.append({"Unicode":uni,
                         "Character":glyph,
                         "caption":cap,
                         "file_name":f"{uni}/{i:03d}.png"})
        except Exception as e:
            logging.warning(f"skip {f} → {e}")

    # -- synthetic extra (10) --
    j = 0
    while j < 10:
        img = render_font(glyph, RNG.choice(font_paths))
        if img:
            fname = f"font_{j:02d}.png"
            img.save(out_dir/fname)
            meta.append({"Unicode":uni,
                         "Character":glyph,
                         "caption":cap,
                         "file_name":f"{uni}/{fname}"})
            j += 1

In [89]:
# %% ──────────────────────────────────────────────────────────────
MAYAN_DIR = Path("/content/drive/MyDrive/EMNLP/mayan_glyphs_100")   # ← upload your images here first
if MAYAN_DIR.exists():
    uni     = "XXXX"             # change to actual code-point if known
    caption = "MAM" # https://mayaglyphs.org/ 1518 MAM or grandfather
    for i, p in enumerate(sorted(MAYAN_DIR.iterdir())[:KEEP_N]):
        fname = f"{i:03d}.png"
        (OUT_ROOT/uni).mkdir(parents=True, exist_ok=True)
        shutil.copy2(p, OUT_ROOT/uni/fname)
        meta.append({"Unicode":uni,
                     "Character":"MAM_Grandfather",
                     "caption":caption,
                     "file_name":f"{uni}/{fname}"})

In [90]:
# %% ──────────────────────────────────────────────────────────────
meta_path = OUT_ROOT/"metadata.jsonl"
with meta_path.open("w", encoding="utf-8") as fh:
    for rec in meta:
        fh.write(json.dumps(rec, ensure_ascii=False) + "\n")

print(f"✅  wrote {len(meta):,} lines → {meta_path}")

✅  wrote 1,000 lines → data/metadata.jsonl


In [85]:
# %% ──────────────────────────────────────────────────────────────
from collections import Counter, defaultdict
cnt = Counter(rec["caption"] for rec in meta)
print("\nImages per caption:")
for k,v in cnt.items():
    print(f"{k:<5}: {v}")

subdirs = defaultdict(int)
for rec in meta:
    subdirs[rec["Unicode"]] += 1
assert all(v==cnt[rec['caption']] for rec,v in subdirs.items() if rec in subdirs)


Images per caption:
A    : 100
B    : 100
G    : 100
ka   : 100
la   : 100
om   : 100
faa  : 100
qaaf : 100
meem : 100
MAM  : 100


TypeError: string indices must be integers, not 'str'

# T5

In [80]:
import torch
import transformers
from typing import List
from transformers import T5Tokenizer, T5EncoderModel, T5Config
from einops import rearrange

transformers.logging.set_verbosity_error()

def exists(val):
    return val is not None

def default(val, d):
    if exists(val):
        return val
    return d() if callable(d) else d

# config

MAX_LENGTH = 256

DEFAULT_T5_NAME = 'google/t5-v1_1-base'

T5_CONFIGS = {}

# singleton globals

def get_tokenizer(name):
    tokenizer = T5Tokenizer.from_pretrained(name, model_max_length=MAX_LENGTH)
    return tokenizer

def get_model(name):
    model = T5EncoderModel.from_pretrained(name)
    return model

def get_model_and_tokenizer(name):
    global T5_CONFIGS

    if name not in T5_CONFIGS:
        T5_CONFIGS[name] = dict()
    if "model" not in T5_CONFIGS[name]:
        T5_CONFIGS[name]["model"] = get_model(name)
    if "tokenizer" not in T5_CONFIGS[name]:
        T5_CONFIGS[name]["tokenizer"] = get_tokenizer(name)

    return T5_CONFIGS[name]['model'], T5_CONFIGS[name]['tokenizer']

def get_encoded_dim(name):
    if name not in T5_CONFIGS:
        # avoids loading the model if we only want to get the dim
        config = T5Config.from_pretrained(name)
        T5_CONFIGS[name] = dict(config=config)
    elif "config" in T5_CONFIGS[name]:
        config = T5_CONFIGS[name]["config"]
    elif "model" in T5_CONFIGS[name]:
        config = T5_CONFIGS[name]["model"].config
    else:
        assert False
    return config.d_model

# encoding text

def t5_tokenize(
    texts: List[str],
    name = DEFAULT_T5_NAME
):
    t5, tokenizer = get_model_and_tokenizer(name)

    if torch.cuda.is_available():
        t5 = t5.cuda()

    device = next(t5.parameters()).device

    encoded = tokenizer.batch_encode_plus(
        texts,
        return_tensors = "pt",
        padding = 'longest',
        max_length = MAX_LENGTH,
        truncation = True
    )

    input_ids = encoded.input_ids.to(device)
    attn_mask = encoded.attention_mask.to(device)
    return input_ids, attn_mask

def t5_encode_tokenized_text(
    token_ids,
    attn_mask = None,
    pad_id = None,
    name = DEFAULT_T5_NAME
):
    assert exists(attn_mask) or exists(pad_id)
    t5, _ = get_model_and_tokenizer(name)

    attn_mask = default(attn_mask, lambda: (token_ids != pad_id).long())

    t5.eval()

    with torch.no_grad():
        output = t5(input_ids = token_ids, attention_mask = attn_mask)
        encoded_text = output.last_hidden_state.detach()

    attn_mask = attn_mask.bool()

    encoded_text = encoded_text.masked_fill(~rearrange(attn_mask, '... -> ... 1'), 0.) # just force all embeddings that is padding to be equal to 0.
    return encoded_text

def t5_encode_text(
    texts: List[str],
    name = DEFAULT_T5_NAME,
    return_attn_mask = False
):
    token_ids, attn_mask = t5_tokenize(texts, name = name)
    encoded_text = t5_encode_tokenized_text(token_ids, attn_mask = attn_mask, name = name)

    if return_attn_mask:
        attn_mask = attn_mask.bool()
        return encoded_text, attn_mask

    return encoded_text


# Utils

In [81]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms as T
from torch.nn.utils.rnn import pad_sequence

from PIL import Image, ImageDraw, ImageFont

from diffusers.pipelines.pipeline_utils import DiffusionPipeline, ImagePipelineOutput
from typing import List, Optional, Tuple, Union
from diffusers.utils.torch_utils import randn_tensor


# Collator adjusted for local dataset
class Collator:
    def __init__(self, image_size, text_label, image_label, name, channels):
        self.text_label = text_label
        self.image_label = image_label
        self.name = name
        self.channels = channels
        self.transform = T.Compose([
            T.Resize((image_size, image_size)),
            T.ToTensor(),
        ])

    def __call__(self, batch):
        texts = []
        masks = []
        images = []
        for item in batch:
            try:
                # Load image from local file
                image_path = 'data/'+item[self.image_label]  # Assuming this is a path to the image file
                with Image.open(image_path) as img:
                    image = self.transform(img.convert(self.channels))
            except Exception as e:
                print(f"Failed to process image {image_path}: {e}")
                continue

            # Encode the text
            text, mask = t5_encode_text(
                [item[self.text_label]],
                name=self.name,
                return_attn_mask=True
                )
            texts.append(torch.squeeze(text))
            masks.append(torch.squeeze(mask))
            images.append(image)

        if len(texts) == 0:
            return None

        # Are these strictly necessary?
        texts = pad_sequence(texts, True)
        masks = pad_sequence(masks, True)

        newbatch = []
        for i in range(len(texts)):
            newbatch.append((images[i], texts[i], masks[i]))

        return torch.utils.data.dataloader.default_collate(newbatch)


class GlyffuserPipeline(DiffusionPipeline):
    r'''
    Pipeline for text-to-image generation from the glyffuser model

    Parameters:
        unet (['UNet2DConditionModel'])
        scheduler (['SchedulerMixin'])
        text_encoder (['TextEncoder']) - T5 small
    '''
    def __init__(self, unet, scheduler):
        super().__init__()
        self.register_modules(
            unet=unet,
            scheduler=scheduler,
            )

    @torch.no_grad()
    def __call__(
        self,
        texts: List[str],
        text_encoder: str = "google-t5/t5-small",
        batch_size: int = 1,
        generator: Optional[Union[torch.Generator, List[torch.Generator]]] = None,
        num_inference_steps: int = 1000,
        output_type: Optional[str] = "pil",
        return_dict: bool = True,
    ) -> Union[ImagePipelineOutput, Tuple]:
        '''
        Docstring
        '''
        # Get text embeddings
        # Encode the text
        # text_embeddings = []
        # for text in texts:
        #     embedding = t5.t5_encode_text(text, name=text_encoder)
        #     text_embeddings.append(torch.squeeze(embedding))
        # text_embeddings = pad_sequence(text_embeddings, True)

        batch_size = len(texts)

        text_embeddings, masks = t5_encode_text(texts, name=text_encoder, return_attn_mask=True)

        # Sample gaussian noise to begin loop
        if isinstance(self.unet.config.sample_size, int):
            image_shape = (
                batch_size,
                self.unet.config.in_channels,
                self.unet.config.sample_size,
                self.unet.config.sample_size,
            )
        else:
            image_shape = (batch_size, self.unet.config.in_channels, *self.unet.config.sample_size)


        # if self.device.type == "mps": # MPS is apple silicon
        #     # randn does not work reproducibly on mps
        #     image = randn_tensor(image_shape, generator=generator)
        #     image = image.to(self.device)
        # else:
        image = randn_tensor(image_shape, generator=generator, device=self.device)

        # set step values
        self.scheduler.set_timesteps(num_inference_steps)

        for t in self.progress_bar(self.scheduler.timesteps):
            # 1. predict noise model_output
            model_output = self.unet(
                image,
                t,
                encoder_hidden_states=text_embeddings, # Add text encoding input
                encoder_attention_mask=masks, # Add attention mask
                return_dict=False
                )[0] # <-- sample is an attribute of the BaseOutClass of type torch.FloatTensor

            # 2. compute previous image: x_t -> x_t-1
            image = self.scheduler.step(model_output, t, image, generator=generator, return_dict=False)[0]

        # image = (image / 2 + 0.5).clamp(0, 1)
        image = image.clamp(0, 1) # No need to rescale for HF yuewu/glyffuser
        image = image.cpu().permute(0, 2, 3, 1).numpy()
        if output_type == "pil":
            image = self.numpy_to_pil(image)

        if not return_dict:
            return (image,)

        return ImagePipelineOutput(images=image)

def make_grid(images, rows, cols):
    w, h = images[0].size
    grid = Image.new('RGB', size=(cols*w, rows*h))
    for i, image in enumerate(images):
        grid.paste(image, box=(i%cols*w, i//cols*h))
    return grid

def evaluate(config, epoch, texts, pipeline):
    images = pipeline(
        texts,
        batch_size = config.eval_batch_size,
        generator=torch.Generator(device='cpu').manual_seed(config.seed), # Generator must be on CPU for sampling during training
    ).images

    # Make a grid out of the images
    image_grid = make_grid(images, rows=4, cols=4)

    # Save the images
    test_dir = os.path.join(config.output_dir, "samples")
    os.makedirs(test_dir, exist_ok=True)
    image_grid.save(f"{test_dir}/{epoch:04d}.png")

def make_labeled_grid(images, prompt, steps, font_path=None, font_size=20, margin=10):
    assert len(images) == len(steps), "The number of images must match the number of steps"

    w, h = images[0].size
    font = ImageFont.truetype(font_path, font_size) if font_path else ImageFont.load_default()

    # Calculate the height of the grid including the margin for text
    total_height = h + margin + font_size
    total_width = w * len(images)
    grid_height = total_height + margin + font_size  # Add extra margin for the prompt
    grid = Image.new('RGB', size=(total_width, grid_height), color=(255, 255, 255))

    # Draw the text prompt at the top
    draw = ImageDraw.Draw(grid)
    prompt_text = f"Prompt: \"{prompt}\""
    prompt_width, prompt_height = draw.textbbox((0, 0), prompt_text, font=font)[2:4]
    prompt_x = (total_width - prompt_width) / 2
    prompt_y = margin / 2
    draw.text((prompt_x, prompt_y), prompt_text, fill="black", font=font)

    for i, (image, step) in enumerate(zip(images, steps)):
        # Calculate position to paste the image
        x = i * w
        y = margin + font_size

        # Paste the image
        grid.paste(image, box=(x, y))

        # Draw the step text
        step_text = f"Steps: {step}"
        text_width, text_height = draw.textbbox((0, 0), step_text, font=font)[2:4]
        text_x = x + (w - text_width) / 2
        text_y = y + h + margin / 2 - 8
        draw.text((text_x, text_y), step_text, fill="black", font=font)

    return grid

# Conditional Glyffuser

### Define training parameters
We add text_encoder and encoder_dim parameters to the config. The batch size is also smaller due to the larger size of the conditional model.

In [82]:
import os, numpy as np, matplotlib.pyplot as plt, torch, torch.nn.functional as F
from torch.utils.data import DataLoader
import torch.nn.functional as F
from datasets import load_dataset
from diffusers.optimization import get_cosine_schedule_with_warmup
from diffusers import UNet2DConditionModel, DDPMScheduler
from accelerate import Accelerator
from tqdm.auto import tqdm
import os
from torchvision.utils import save_image
import numpy as np
import matplotlib.pyplot as plt
from dataclasses import dataclass
from diffusers import DPMSolverMultistepScheduler
from torchmetrics.image.fid import FrechetInceptionDistance

In [91]:
@dataclass
class TrainingConfig:
    image_size = 128  # the generated image resolution
    train_batch_size = 16
    eval_batch_size = 16  # how many images to sample during evaluation
    num_epochs = 200
    gradient_accumulation_steps = 1
    learning_rate = 1e-4
    lr_warmup_steps = 500
    save_image_epochs = 25
    save_model_epochs = 30
    mixed_precision = "fp16"  # `no` for float32, `fp16` for automatic mixed precision
    output_dir = "/content/drive/MyDrive/EMNLP/multilingual_diffuser/test"
    overwrite_output_dir = True  # overwrite the old model when re-running the notebook
    text_encoder = "google-t5/t5-small"
    encoder_dim = 512
    seed = 0

config = TrainingConfig()

In [92]:
import torch
from torch.utils.data import DataLoader
import torch.nn.functional as F
from datasets import load_dataset
from diffusers.optimization import get_cosine_schedule_with_warmup
from diffusers import UNet2DConditionModel, DDPMScheduler
from accelerate import Accelerator
from tqdm.auto import tqdm
import os
from torchvision.utils import save_image  # optional helper
import numpy as np
import matplotlib.pyplot as plt


def train_loop(config, model, noise_scheduler, optimizer, train_dataloader, lr_scheduler):
    # Initialize accelerator and tensorboard logging
    accelerator = Accelerator(
        mixed_precision=config.mixed_precision,
        gradient_accumulation_steps=config.gradient_accumulation_steps,
        log_with="tensorboard",
        project_dir=os.path.join(config.output_dir, "logs")
    )
    if accelerator.is_main_process:

        if config.output_dir is not None:
            os.makedirs(config.output_dir, exist_ok=True)
        sample_dir = os.path.join(config.output_dir, "samples")
        os.makedirs(sample_dir, exist_ok=True)
        accelerator.init_trackers("train_example")

    # Prepare everything for accelerator
    model, optimizer, train_dataloader, lr_scheduler = accelerator.prepare(
        model, optimizer, train_dataloader, lr_scheduler
    )

    global_step = 0
    loss_hist, kl_hist = [], []

    # Now you train the model
    for epoch in range(config.num_epochs):
        progress_bar = tqdm(total=len(train_dataloader), disable=not accelerator.is_local_main_process)
        progress_bar.set_description(f"Epoch {epoch}")

        for step, batch in enumerate(train_dataloader):
            clean_images = batch[0]
            noise = torch.randn(clean_images.shape).to(clean_images.device)
            bs = clean_images.shape[0]
            timesteps = torch.randint(0, noise_scheduler.num_train_timesteps, (bs,), device=clean_images.device).long()
            noisy_images = noise_scheduler.add_noise(clean_images, noise, timesteps)
            with accelerator.accumulate(model):
                noise_pred = model(
                    noisy_images,
                    timesteps,
                    encoder_hidden_states=batch[1],
                    return_dict=False
                    )[0]
                loss = F.mse_loss(noise_pred, noise)

                # --- KL divergence between N(noise_pred, I) and N(0, I)
                kl_loss = 0.5 * (noise_pred - noise).pow(2).mean()   # <── NEW
                loss_hist.append(loss.item())                        # <── NEW
                kl_hist.append(kl_loss.item())

                accelerator.backward(loss)
                accelerator.clip_grad_norm_(model.parameters(), 1.0)
                optimizer.step()
                lr_scheduler.step()
                optimizer.zero_grad()

            progress_bar.update(1)

            logs = {
                  "loss": loss.item(),
                  "kl_loss": kl_loss.item(),       # <── NEW (TensorBoard)
                  "lr": lr_scheduler.get_last_lr()[0],
                  "step": global_step,
                    }

            progress_bar.set_postfix(**logs)
            accelerator.log(logs, step=global_step)
            global_step += 1


        if accelerator.is_main_process and (
            (epoch + 1) % config.save_image_epochs == 0
            or epoch == config.num_epochs - 1
        ):
            pipe = GlyffuserPipeline(
                unet=accelerator.unwrap_model(model),
                scheduler=inference_scheduler
            ).to(accelerator.device)

            texts = [
                      "A", "B", "G",          # Latin
                      "faa", "qaaf", "meem",  # Arabic
                      "ka", "la", "ga",        # Devanagari
                      "MAM"
                    ]
            pipe.scheduler.set_timesteps(25)
            images = [pipe(t, num_inference_steps=25).images[0] for t in texts]

            # ----- make grid & save individual PNGs -----
            stack = []
            for cap, im in zip(texts, images):
                arr = np.array(im)
                if arr.ndim == 2:          # grayscale -> (H,W,1)
                    arr = arr[..., None]
                tensor = torch.from_numpy(arr).permute(2, 0, 1).float() / 255
                stack.append(tensor)
                im.save(os.path.join(sample_dir, f"epoch_{epoch:03d}_{cap}.png"))

            grid = torch.stack(stack)      # (N,C,H,W)
            save_image(grid,
                       os.path.join(sample_dir, f"grid_{epoch:03d}.png"),
                       nrow=len(texts))

            # save checkpoint
            pipe.save_pretrained(os.path.join(
                config.output_dir, f"checkpoint-{epoch:03d}"
            ))
            del pipe

    # ─── final loss curves ─────────────────────────────────────────
    if accelerator.is_main_process:
        plt.figure()
        plt.plot(loss_hist, label="MSE loss")
        plt.plot(kl_hist,   label="KL loss (½·MSE)")
        plt.yscale("log")
        plt.xlabel("training step")
        plt.ylabel("loss")
        plt.legend()
        plot_path = os.path.join(config.output_dir, "loss_curves_final.png")
        plt.savefig(plot_path, dpi=150, bbox_inches="tight")
        plt.close()
        print(f"Saved final loss curves → {plot_path}")


We add a collator which helps the dataloader pass text embeddings to the model during training

In [93]:
from diffusers import DPMSolverMultistepScheduler

# Define collator
collator = Collator(
    image_size=128,
    text_label='caption',  # This is where you specify which field to use for text
    image_label='file_name',
    name=config.text_encoder,
    channels='L'
)

# Define data source
dataset = load_dataset("json", data_files="data/metadata.jsonl")
train_dataloader = DataLoader(dataset['train'], batch_size=config.train_batch_size, collate_fn=collator, shuffle=True)

# Define model
model = UNet2DConditionModel(
    sample_size=config.image_size,  # the target image resolution
    in_channels=1,  # the number of input channels, 1 for RGB images
    out_channels=1,  # the number of output channels
    layers_per_block=2,  # how many ResNet layers to use per UNet block
    # block_out_channels=(128, 128, 256, 256, 512, 512),  # the number of output channels for each UNet block
    block_out_channels=(128, 256, 512, 512),  # the number of output channels for each UNet block

    addition_embed_type="text", # Make it conditional
    cross_attention_dim=config.encoder_dim,
    encoder_hid_dim=config.encoder_dim,  # the hidden dimension of the encoder
    encoder_hid_dim_type="text_proj",  # the hidden dimension of the encoder
    down_block_types=(
        "DownBlock2D",
        "DownBlock2D",
        "CrossAttnDownBlock2D",
        "DownBlock2D"
    ),
    up_block_types=(
        "UpBlock2D",
        "CrossAttnUpBlock2D",
        "UpBlock2D",
        "UpBlock2D"
    ),
)

# Define optimizers and schedulers
noise_scheduler = DDPMScheduler(num_train_timesteps=1000)
inference_scheduler = DPMSolverMultistepScheduler()
optimizer = torch.optim.AdamW(model.parameters(), lr=config.learning_rate)
lr_scheduler = get_cosine_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=config.lr_warmup_steps,
    num_training_steps=(len(train_dataloader) * config.num_epochs),
)
# Check model parameters
total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'{total_params:,} total parameters.')

Generating train split: 0 examples [00:00, ? examples/s]

130,790,657 total parameters.


Pass everything to accelerate and run the training loop

In [94]:
from accelerate import notebook_launcher

args = (config, model, noise_scheduler, optimizer, train_dataloader, lr_scheduler)
notebook_launcher(train_loop, args, num_processes=1)

Launching training on one GPU.


  0%|          | 0/63 [00:00<?, ?it/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/diffusers/configuration_utils.py:141: FutureWarning: Accessing config attribute `num_train_timesteps` directly via 'DDPMScheduler' object attribute is deprecated. Please access 'num_train_timesteps' over 'DDPMScheduler's config object instead, e.g. 'scheduler.config.num_train_timesteps'.
  deprecate("direct config name access", "1.0.0", deprecation_message, standard_warn=False)


KeyboardInterrupt: 

In [ ]:
from diffusers import DPMSolverMultistepScheduler
import torch

# Initialize the model pipeline
pipeline = GlyffuserPipeline.from_pretrained(f"{config.output_dir}/checkpoint-199").to("cuda")
pipeline.scheduler = DPMSolverMultistepScheduler()

texts = ['X']  # Your text prompt
num_inference_steps_list = [1, 2, 3, 5, 10, 20, 50, 100]  # List of different num_inference_steps values

images = []

# Generate images for each value in num_inference_steps_list
for num_steps in num_inference_steps_list:
    generated_images = pipeline(
        texts,
        batch_size=1,  # Generate one image at a time for each step
        generator=torch.Generator(device='cuda').manual_seed(42),  # Generator can be on GPU here
        num_inference_steps=num_steps
    ).images
    images.append(generated_images[0])  # Append the generated image

# Create the labeled grid
image_grid = make_labeled_grid(images, texts[0], num_inference_steps_list, font_size=30, margin=20)
image_grid

# Save the image grid
# image_grid.save(f"inference example.png") # Uncomment to save

In [ ]:
from pathlib import Path
import torch, numpy as np, math
from diffusers import DPMSolverMultistepScheduler
from PIL import Image, ImageDraw, ImageFont

# ───────────────── helper: labelled grid ──────────────────────────
def make_labeled_grid(images, caption, steps, font_size=24, margin=8):
    """Return a PIL.Image that shows `images` in a row with step labels."""
    w, h = images[0].size
    font = ImageFont.load_default()
    total_w = w * len(images)
    total_h = h + font_size + margin

    grid = Image.new("RGB", (total_w, total_h), "white")
    draw = ImageDraw.Draw(grid)

    for i, (img, s) in enumerate(zip(images, steps)):
        grid.paste(img, (i * w, 0))
        txt = f"{s}"
        tw, th = draw.textbbox((0, 0), txt, font=font)[2:]
        tx = i * w + (w - tw) // 2
        draw.text((tx, h + margin // 2), txt, fill="black", font=font)

    # title (caption) centred on top
    title_w, _ = draw.textbbox((0, 0), caption, font=font)[2:]
    draw.text(((total_w - title_w) // 2, -margin // 2), caption, fill="black", font=font)
    return grid

# ───────────────── config  ────────────────────────────────────────
ckpt_dir   = f"{config.output_dir}/checkpoint-199"   # <-- latest trained ckpt
out_root   = Path(config.output_dir) / "inference"
out_root.mkdir(exist_ok=True)

captions   = [          # 15 glyphs (5×Latin,5×Arabic,5×Devanagari)
    "A","B","W","D","G",
    "faa","qaaf","ghayn","laam","meem",
    "ka","la","om","ga","ma"
]
num_steps  = [1, 2, 3, 5, 10, 20, 50, 100]
seed       = 42
device     = "cuda"

# ───────────────── pipeline  ──────────────────────────────────────
pipe = GlyffuserPipeline.from_pretrained(ckpt_dir).to(device)
pipe.scheduler = DPMSolverMultistepScheduler()

# ───────────────── generation  ────────────────────────────────────
mega_rows = []                # for optional mega-grid
for cap in captions:
    subdir = out_root / cap.replace(" ", "_")
    subdir.mkdir(exist_ok=True)

    imgs_for_grid = []
    for n in num_steps:
        img = pipe(
            cap,
            num_inference_steps=n,
            batch_size=1,
            generator=torch.Generator(device).manual_seed(seed),
        ).images[0]
        img.save(subdir / f"{cap}_{n:03}.png")
        imgs_for_grid.append(img)

    # grid for this caption
    g = make_labeled_grid(imgs_for_grid, cap, num_steps)
    g.save(subdir / f"{cap}_grid.png")
    mega_rows.append(imgs_for_grid[-1])     # take the *final* step for mega grid

# ───────────────── optional mega-grid (all captions)  ─────────────
cols = 5
rows = math.ceil(len(mega_rows) / cols)
w, h = mega_rows[0].size
mega = Image.new("RGB", (w * cols, h * rows), "white")
for idx, img in enumerate(mega_rows):
    r, c = divmod(idx, cols)
    mega.paste(img, (c * w, r * h))
mega.save(out_root / "mega_grid.png")

print("✅  Inference images saved to:", out_root)